In [ ]:
%%time
print("starting")
# from checkra import happyscribe_scrape as happy

main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"

# happy.update_transcripts(html_location, main)

In [ ]:
%%time
from checkra.text_clean import text_fix, strip_accents

import spacy
import os
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import re
from checkra.nlp_pipeline import nlp

# doc = nlp(text_fix(open("3Lex/#109|Brian_Kernighan|UNIX,_C,_AWK,_AMPL,_and_Go_Programming.txt").read()))

# print(doc.user_data.keys())


In [ ]:
%%time

print("starting")
folder="3Lex"
podcast_host = "Lex Fridman"


# docs, doc_bin = process_folder_to_docs("5Lex", podcast_host)
onlyfiles=[(text_fix(open(folder+"/"+file).read()), file) for file in listdir(folder) if isfile(join(folder, file))]
docs = [] #list of docs 

for doc, name in tqdm(nlp.pipe(onlyfiles, as_tuples=True)): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    doc.user_data["url"] = main+url+"/"+strip_accents(name.replace("|","-").replace("#","").replace(".txt","").replace("_","-").replace(",","").lower())
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    docs.append(doc)

print("done")


In [ ]:
for ind, doc in enumerate(docs):
    print(ind, doc.user_data["guest"], doc.user_data["title"])

In [ ]:
docs[0].user_data["traits"].keys()

In [ ]:
import shutil
count = 0
not_processed = []
for ind, file in enumerate(os.listdir("7Lex")):
    if file.endswith(".txt"):
        file2 = file.replace("_", " ").replace("|", " ").replace(".txt", " ")
    #     print(ind, file) 
        processed = False


        for doc in docs:
            if doc.user_data["guest"] in file2 and doc.user_data["title"] in file2:
    #             os.rename("5Lex/"+file, "7Lex/"+file)
                
                processed = True
                count+=1
                break
        if processed:
            os.rename("7Lex/"+file, "5Lex/"+file)
print("done")

In [ ]:
import pymongo
from pymongo import MongoClient

username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["production"]
collection = db["lex"]

# db.create_collection("lmini")

In [ ]:
queried = list(db["lex"].find({},{"_id":0,"traits.Events":1}))
all_ents = [ent for doc in queried for ent in doc["traits"]["Events"]]

In [ ]:
db["lex"].drop()
db["lex"].insert_many([doc.user_data for doc in docs])

In [ ]:
all_atts = []
for doc in list(collection.find({"books":{"$ne":[]}})):
    print(doc["guest"], doc[att])
    for a in doc[att]:
        all_atts.append(a)

In [ ]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
doc_bin = DocBin(store_user_data=True) #docbin container for serialization

    doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())

with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))

In [ ]:

import networkx as nx
import pandas as pd
att_to_csv(docs1, "people") #to get csv of edges

#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()